In [80]:
%%markdown
Using Beautiful soup the content of the Wikipedia page can be obtained as lxml.
Soup can find the tables, in our case as seen, the firs element of a list of tables contains the neighbourhoods.

Using Beautiful soup the content of the Wikipedia page can be obtained as lxml.
Soup can find the tables, in our case as seen, the firs element of a list of tables contains the neighbourhoods.


In [85]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.select_one('table')

df = pd.read_html(str(table))
neighborhouds = df[0]
neighborhouds.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [83]:
%%markdown
Get rid of rows where borough is 'Not assigned'

Get rid of rows where borough is 'Not assigned'


In [84]:
eligible_ns = neighborhouds[(neighborhouds.Borough != "Not assigned")]
eligible_ns.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [ ]:
When same postcodes appear in multiple rows, merge the rows and list all neighbourhoods

In [66]:
# create new df, add first row
combined_postcode_ns = eligible_ns[0:1]
# loop through rest of old df
for i in range (1, len(eligible_ns)):
    # if postcode not yet part of new df, add row from old df
    
    if len(combined_postcode_ns[combined_postcode_ns.Postcode == eligible_ns.iloc[i][0]]) == 0:
        #rint(combined_postcode_ns)
        #rint(eligible_ns[i:i+1])
        combined_postcode_ns = combined_postcode_ns.append( eligible_ns[i:i+1] )
        
    else:
        pass
        # if postcode already part of new df, keep its neighbourhood and concatenate value from old df
        # index of row
        row_index = combined_postcode_ns[(combined_postcode_ns.Postcode == eligible_ns.iloc[i][0])].index
        existing_value = combined_postcode_ns.loc[row_index, 'Neighbourhood']
        new_value = existing_value + ", " + eligible_ns.iloc[i][2]
        # update cell
        combined_postcode_ns.loc[row_index, 'Neighbourhood'] = new_value
    # end if
# end for
combined_postcode_ns.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


In [86]:
%%markdown
If a neighbourhood is still not assigned, should be replace by borough
Note:
The task is executed in the order it was requested, thus if a neighbourhood contains 'Not assigned' in conjuction with another neighbourhood, will not be replaced.
There is no clarity in the instructions whether it should be/should have been done, though fairly simple to do so.

If a neighbourhood is still not assigned, should be replace by borough
Note:
The task is executed in the order it was requested, thus if a neighbourhood contains 'Not assigned' in conjuction with another neighbourhood, will not be replaced.
There is no clarity in the instructions whether it should be/should have been done, though fairly simple to do so.


In [79]:
for i in range(0, len(combined_postcode_ns)):
    #print(combined_postcode_ns.iloc[i][2])
    if combined_postcode_ns.iloc[i][2] == 'Not assigned':
        combined_postcode_ns.iloc[i][2] = combined_postcode_ns.iloc[i][1]
    # end if
# end for

combined_postcode_ns.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,"Rouge, Malvern"
13,M3B,North York,Don Mills North
14,M4B,East York,"Woodbine Gardens, Parkview Hill"
16,M5B,Downtown Toronto,"Ryerson, Garden District"


In [92]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
combined_postcode_nswg = combined_postcode_ns
combined_postcode_nswg = combined_postcode_nswg.join(df_data_1.set_index('Postal Code'), on='Postcode')
combined_postcode_nswg.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
7,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
10,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
13,M3B,North York,Don Mills North,43.745906,-79.352188
14,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
16,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
